In [1]:
import pandas as pd 
import numpy as np

from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb


In [2]:
import os
for dirname, _, filenames in os.walk('C:\\Eric\\Data\\M5'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Eric\Data\M5\SNAP_CA.csv
C:\Eric\Data\M5\SNAP_TX.csv
C:\Eric\Data\M5\SNAP_WI.csv


In [3]:
SNAP_pl_ca = pd.read_csv("C:\Eric\Data\M5\SNAP_CA.csv")
SNAP_pl_tx = pd.read_csv("C:\Eric\Data\M5\SNAP_TX.csv")
SNAP_pl_wi = pd.read_csv("C:\Eric\Data\M5\SNAP_WI.csv")

In [4]:
SNAP_pl_ca.dtypes

DATE              object
BRCA06M647NCEN     int64
dtype: object

In [5]:
import os
for dirname, _, filenames in os.walk('C:\Eric\Data\m5-forecasting-accuracy'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Eric\Data\m5-forecasting-accuracy\calendar.csv
C:\Eric\Data\m5-forecasting-accuracy\M5_codes.txt
C:\Eric\Data\m5-forecasting-accuracy\sales_train_evaluation.csv
C:\Eric\Data\m5-forecasting-accuracy\sales_train_validation.csv
C:\Eric\Data\m5-forecasting-accuracy\sample_submission.csv
C:\Eric\Data\m5-forecasting-accuracy\sell_prices.csv


In [6]:
# 주요 데이터 불러오기 
calendar         = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\calendar.csv")
train_validation = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sales_train_validation.csv")
sell_prices = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sell_prices.csv")

In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [11]:
calendar = reduce_mem_usage(calendar)
train_validation = reduce_mem_usage(train_validation)
sell_prices = reduce_mem_usage(sell_prices)

Memory usage after optimization is: 0.12 MB
Decreased by 41.9%
Memory usage after optimization is: 95.00 MB
Decreased by 78.7%
Memory usage after optimization is: 130.48 MB
Decreased by 37.5%


In [ ]:
# SNAP_pl_ca = pd.read_csv("C:\Eric\Data\M5\SNAP_CA.csv")
# SNAP_pl_tx = pd.read_csv("C:\Eric\Data\M5\SNAP_TX.csv")
# SNAP_pl_wi = pd.read_csv("C:\Eric\Data\M5\SNAP_WI.csv")
# 주요 데이터 불러오기 
# calendar         = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\calendar.csv")
# train_validation = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sales_train_validation.csv")
# sell_prices = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sell_prices.csv")

In [11]:
# To create "month" and "year" columns in SNAP recipient data. 

SNAP_pl_ca.columns = ['date', 'recipient']
SNAP_pl_wi.columns = ['date', 'recipient']
SNAP_pl_tx.columns = ['date', 'recipient']

SNAP_pl_ca['date'] = pd.to_datetime(SNAP_pl_ca.date, errors='coerce')
SNAP_pl_wi['date'] = pd.to_datetime(SNAP_pl_wi.date, errors='coerce')
SNAP_pl_tx['date'] = pd.to_datetime(SNAP_pl_tx.date, errors='coerce')

# date col에서 month 뽑아내기 
SNAP_pl_ca['date_month'] = SNAP_pl_ca['date'].dt.strftime('%m')
SNAP_pl_wi['date_month'] = SNAP_pl_wi['date'].dt.strftime('%m')
SNAP_pl_tx['date_month'] = SNAP_pl_tx['date'].dt.strftime('%m')

# date col에서 year 뽑아내기 
SNAP_pl_ca['date_year'] = SNAP_pl_ca['date'].dt.strftime('%Y')
SNAP_pl_wi['date_year'] = SNAP_pl_wi['date'].dt.strftime('%Y')
SNAP_pl_tx['date_year'] = SNAP_pl_tx['date'].dt.strftime('%Y')

In [19]:
# CA type change 
SNAP_pl_ca.date_month = pd.to_numeric(SNAP_pl_ca.date_month)
SNAP_pl_ca.date_year = pd.to_numeric(SNAP_pl_ca.date_year)
# WI type change 
SNAP_pl_ca.date_month = pd.to_numeric(SNAP_pl_wi.date_month)
SNAP_pl_ca.date_year = pd.to_numeric(SNAP_pl_wi.date_year)
# TX type change 
SNAP_pl_ca.date_month = pd.to_numeric(SNAP_pl_tx.date_month)
SNAP_pl_ca.date_year = pd.to_numeric(SNAP_pl_tx.date_year)

# column name change 

SNAP_pl_ca = SNAP_pl_ca.rename(columns = {'date_month':'month'} )
SNAP_pl_wi = SNAP_pl_wi.rename(columns = {'date_month':'month'} )
SNAP_pl_tx = SNAP_pl_tx.rename(columns = {'date_month':'month'} )

SNAP_pl_ca = SNAP_pl_ca.rename(columns = {'date_year':'year'} )
SNAP_pl_wi = SNAP_pl_wi.rename(columns = {'date_year':'year'} )
SNAP_pl_tx = SNAP_pl_tx.rename(columns = {'date_year':'year'} )

# type changer *(aboce code didn't work)
SNAP_pl_wi.month = pd.to_numeric(SNAP_pl_wi.month)
SNAP_pl_wi.year = pd.to_numeric(SNAP_pl_wi.year)
SNAP_pl_tx.month = pd.to_numeric(SNAP_pl_tx.month)
SNAP_pl_tx.year = pd.to_numeric(SNAP_pl_tx.year)

In [24]:
cal01 = calendar.copy()
cal01.shape

(1969, 14)

In [25]:
SNAP_pl_ca.shape

(450, 4)

In [27]:
# SNAP_pl_ca 에 있는 recipient 숫자를 "year" & "month" 기준으로 cal01 데이터에 merge. 
merge001 = pd.merge(cal01, SNAP_pl_ca, on =["month","year"], how="left"  )

In [34]:
merge001.shape

(1969, 16)

In [39]:
# 각 주의 recipient 숫자를 calendar 데이터에 merge 하기, (on "year" & "month")
# merge 방법 간략화 하기 참조 

#     dt = pd.melt(dt,
#                   id_vars = catcols,
#                   value_vars = [col for col in dt.columns if col.startswith("d_")],
#                   var_name = "d",
#                   value_name = "sales")
    
#     dt = dt.merge(cal, on= "d", copy = False)
#     dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

merge001 = merge001.rename(columns = {'recipient':'ca_recipient'} )

In [48]:
merge_f = pd.merge(merge001, SNAP_pl_wi, on =["month","year"], how="left"  )

In [49]:
merge_f = merge_f.rename(columns = {'recipient':'wi_recipient'} )

In [51]:
merge_x = pd.merge(merge_f, SNAP_pl_tx, on = ["month", "year"], how = "left")

In [52]:
merge_x = merge_x.rename(columns = {'recipient':'tx_recipient'} )

In [53]:
merge_x.columns

Index(['date_x', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'date_y', 'ca_recipient', 'date_x',
       'wi_recipient', 'date_y', 'tx_recipient'],
      dtype='object')

In [65]:
train_validation.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1919)

In [57]:
merge_x.head()

,date_x,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,date_y,ca_recipient,date_x,wi_recipient,date_y,tx_recipient
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,2011-01-01,3602170,2011-01-01,787412,2011-01-01,3928058
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,2011-01-01,3602170,2011-01-01,787412,2011-01-01,3928058
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,2011-01-01,3602170,2011-01-01,787412,2011-01-01,3928058
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,2011-02-01,3619055,2011-02-01,793098,2011-02-01,3895618
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,2011-02-01,3619055,2011-02-01,793098,2011-02-01,3895618


In [60]:
train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [61]:
train_validation.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1919)

In [64]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)